### 1. 추천 알고리즘
- 이용자의 성향과 관심사를 분석하고 콘텐츠 이용 행태를 세밀히 관찰하여 개인 맞춤형 콘텐츠를 제공하는 것
- 협업 필터링: 대규모의 기존 사용자 행동 정보를 분석하여 해당 사용자와 비슷한 성향의 사용자들이 기존에 좋아했던 항목을 추천하는 기술
    - 결과가 직관적이며 항목의 구체적인 내용을 분석할 필요가 없다는 장점
    - 콜드 스타트 문제 발생: 기존의 자료가 필요하기 때문에 새로 시작할 때 곤란하다는 단점
    - 계산량 많은 알고리즘: 사용자 수 많은 경우 비효율
    - 롱테일 문제: 시스템 항목이 많아도 사용자들은 소수의 인기 있는 항목에만 관심 보임
- 콘텐츠 기반 필터링: 항목 자체를 분석하여 추천 구현
    - 항목을 분석한 프로파일과 사용자의 선호도르 추출한 프로파일을 추출하여 유사성 계산
    - 아이템 분석 알고리즘: 군집분석, 인공신경망 기술 이용
    - 콜드 스타트 해결
    - 다양한 형식의 항목 추천 어려운 단점
- 모델 기반 협력 필터링: 협력 필터링 고도화, 넷플릭스
    - 기존 항목 간 유사성을 단순하게 비교하는 것에서 벗어나 자료 안에 내재한 패턴을 이용하는 기법
    - 주위 정보를 이용해 선호 이유를 유추함으로써 높은 정확도로 항목 추천
필터버블: 추천 시스템은 정보를 추려서 제공하기 때문에 사용자가 전체 정보를 볼 기회를 박탈당할 수 있다는 위험

- 처음 접속한 사용자라면 이 사용자에 대한 아무런 초기 정보가 없는 상황이어서 사용자 기반 추천을 전혀 할 수 없어 콘텐츠 기반 필터링 방식의 추천만 가능한데, 이를 보완하여 처음부터 사용자 특성에 따른 맞춤형 서비스를 제공하기 위해서이다.
사용자 맞춤형 추천을 제공하지 못한다면 모든 초기 사용자에게 같은 화면을 제시할 수밖에 없는데, 이러면 모든 사용자들이 처음 추천받은 콘텐츠 위주로만 시청하게 되어 추천시스템이 빠지기 쉬운 필터 버블의 문제을 야기하거나, 추천의 다양성을 저해할 수 있는 위험이 생긴다.
- 추천시스템은 1. 아이템이 매우 많고, 2. 유저의 취향이 다양할 때 이용

$ pip install implicit 설치

#### 1-1. tsv: Tab-Separated Values 데이터 준비

In [6]:
import pandas as pd
import os

fname = os.getenv('HOME') + '/workplace/aiffel/Exploration/07. Recommend System/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'
col_names = ['user_id', 'artist_MBID', 'artist', 'play']   # 임의로 지정한 컬럼명
data = pd.read_csv(fname, sep='\t', names= col_names)      # sep='\t'로 주어야 tsv를 열 수 있습니다.  
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


In [7]:
# 사용하는 컬럼만 남겨줍니다. artist_MBID 삭제
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [8]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


In [9]:
# 첫 번째 유저가 어떤 아티스트 노래 듣는지 확인
condition = data['user_id']== data.loc[0, 'user_id']
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


#### 1-2. 데이터 탐색
- 유저수, 아티스트수, 인기 많은 아티스트
- 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
- 유저 play 횟수 중앙값에 대한 통계

**pandas.DataFrame.nunique()**
- 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용

In [10]:
# 유저 수
data['user_id'].nunique()

358868

In [11]:
# 아티스트 수
data['artist'].nunique()

291346

In [12]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [13]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [14]:
# 유저별 몇 명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

#### 1-3. 모델 검증을 위한 사용자 초기 정보 세팅
- 사용자 취향과 유사한 아티스트 정보 5개 이상 입력

In [15]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['black eyed peas' , 'maroon5' ,'jason mraz' ,'coldplay' ,'shakira']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['lea']*5, 'artist': my_favorite, 'play':[30]*5})

if not data.isin({'user_id':['lea']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    data = data.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

data.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,lea,black eyed peas,30
1,lea,maroon5,30
2,lea,jason mraz,30
3,lea,coldplay,30
4,lea,shakira,30


#### 1-4. 모델에 활용하기 위한 전처리

In [16]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
artist_to_idx = {v:k for k,v in enumerate(artist_unique)}

In [17]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['lea'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(artist_to_idx['shakira'])

358868
1785


In [18]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing Fail!!')

data

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


#### 1-5. 사용자의 명시적/암묵적 평가
- 사용자의 평점 기록 데이터: Movielens Data
- Explicit Data(명시적 데이터): 유저가 자신의 선호도를 직접 표현한 데이터 (평점 기록, 구독, 차단 데이터, 영화 리뷰)
    - 유용성이 좋으나 데이터를 얻기 힘들다.
- Implicit Datasets: 유저가 간접적으로 선호, 취향을 나타내는 데이터 (검색 기록, 방문 페이지, 구매 내역)
    - 유저가 개인정보제공에 동의만 한다면 자동적으로 수집되기 때문에 수집의 난이도 낮고 활용할 수 있는 데이터량 많음
    - 부정적인 피드백이 없다.
    - 잡음이 많다.
    - 수치는 신뢰도를 말한다.: 반면 명시적 데이터에서 수치는 선혿도를 의미한다.
    - Implicit-feedback Recommender System의 평가는 적절한 방법을 고민해봐야 한다

출처: https://orill.tistory.com/entry/Explicit-vs-Implicit-Feedback-Datasets?category=1066301 [이제 며칠 후엔]

In [19]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play']<2]
one, all_data = len(only_one), len(data)
print(f'{one},{all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740,17535660
Ratio of only_one over all data is 0.84%


#### 우리 모델의 규칙
- 한 번이라도 들었으면 선호한다고 판단한다.
- 많이 재생한 아티스트에 대해 가중치를 주어서 더 확실히 좋아한다고 판단한다.

### 2. Matrix Factorization(MF) 모델
- 행렬분해 모델은 (m,n) 사이즈의 행렬 R을 (m, k)사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현할 수 있음을 의미한다.
- 대체로 k는 m이나 n보다 훨씬 작은 값이기 때문에 계산량 측면으로도 훨씬 유리하다.
- 하지만 아이디어의 단순함에도 불구하고 MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용되는 모델이다.

- MF 모델은 큰 평가행렬 R을 두개의 Feature Matrix P와 Q로 분해한다.
- (m, k) 사이즈의 Feature Matrix P는 k 차원의 벡터를 사용자 수만큼 모아놓은 행렬이다. 그렇다면 아래 그림에서 첫번째 벡터 P0=(1, 0.1)은 바로 빨간 모자를 쓴 첫번째 사용자의 특성(Feature) 벡터가 된다.
같은 방식으로 Q 행렬의 첫번째 벡터 Q0=(0.9, -0.2)는 해리포터 영화의 특성 벡터가 된다. 
- MF 모델은 이 두 벡터를 내적해서 얻어지는 0.88이 바로 R0,0 으로 정의되는 사용자의 영화 선호도로 보는 모델이다.

**모델의 목표**
- 모든 유저와 아이템에 대해 k-dimension의 벡터를 잘 만드는 것
- 벡터를 잘 만드는 기준은 유저i의 벡터(Ui)와 아이템j의 벡터(Ij)를 내적했을 때 유저i가 아이템j에 대해 평가한 수치(Mij)와 비슷한지 이다.
- Ui ⋅ Ij = Mij

#### 2-1. CSR(Compressed Sparse Row) Matrix
- 평가 행렬 용량이 커지는 이유
    - 유저 수 X 아티스트 수만큼의 정보 안에는 유저가 들어보지 않은 아티스트에 대한 정보까지 모두 행렬에 포함되어 계산되기 때문이다.
    - 아무리 많은 아티스트를 들어 본 유저도 수백명을 넘어갈 만큼의 아티스트를 들어보지 못했는데, 총 아티스트는 29만명이 넘기 때문에 평가행렬 내의 대부분의 공간은 0으로 채워집니다. 이런 행렬을 Sparse Matrix라고 한다.
    - 이런 메모리 낭비를 최소화하기 위해서는 유저가 들어본 아티스트에 대해서만 정보만을 저장하면서 전체 행렬 형태를 유추할 수 있는 데이터 구조가 필요하다.
- 좋은 대안: CSR Matrix
    - Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보로만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조
    - csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]., M,N은 matrix의 shape

In [21]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Row format>

#### 2-2. MF 모델 학습하기
- implicit 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지
-  als(AlternatingLeastSquares) 모델: Matrix Factorization에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 AlternatingLeastSquares 방식이 효과적

In [23]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

#### 2-2-1. AlternatingLeastSquares 클래스의 __init__ 파라미터
1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 

2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 

3. use_gpu : GPU를 사용할 것인지 

4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [25]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.int64'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [26]:
# 모델 훈련
als_model.fit(csr_data_transpose)


#### 2-2-2. 살펴보기
모델이 

1) 벡터와 shakira의 벡터를 어떻게 만들고 있는지와 

2) 두 벡터를 곱하면 어떤 값이 나오는지 살펴보기

In [27]:
lea, shakira = user_to_idx['lea'], artist_to_idx['shakira']
lea_vector, shakira_vector = als_model.user_factors[lea], als_model.item_factors[shakira]

print('슝=3')

슝=3


In [28]:
lea_vector

array([ 0.00841874,  0.7570316 ,  1.485143  ,  0.28330085, -0.42913637,
       -1.2653818 , -0.41064847, -0.13876101,  0.04480318, -0.2052408 ,
        1.0783828 ,  0.3087607 ,  0.263398  ,  0.37753662, -0.5079366 ,
       -0.2710364 , -0.66632295,  0.8042157 ,  0.34589937,  0.08726413,
       -0.5395777 ,  0.379727  , -0.00547088,  0.6797303 , -0.6509014 ,
        0.254585  ,  0.14231221, -0.24069425, -1.1969559 ,  0.8614164 ,
       -0.07209604,  1.1649495 ,  0.18710028,  1.6048776 , -0.34425193,
       -0.02939394, -0.3474621 , -1.143099  ,  0.0836055 ,  0.6873183 ,
        0.10133745, -0.65690386,  0.16954659,  0.25594658, -1.102885  ,
        1.224988  , -0.49552125, -0.6119755 ,  0.13058066,  0.5029736 ,
        0.9979836 ,  0.2712289 , -0.3214617 , -0.7438257 , -0.33935067,
        0.5527011 ,  0.31616405,  0.5345603 , -0.18890876,  0.28117633,
       -0.6178092 , -0.23829056, -0.5783588 ,  0.95570177,  0.3916435 ,
        0.48277897,  0.24111667,  0.30127293,  1.6414982 , -0.68

In [29]:
shakira_vector

array([ 0.00311586,  0.00675487,  0.01817887,  0.00885864,  0.01016829,
       -0.00414678, -0.01149488,  0.00217091,  0.01034558,  0.00391518,
        0.01943563,  0.00118207,  0.01149569,  0.01029247,  0.00176776,
       -0.00096722,  0.014333  ,  0.01806135,  0.01549488,  0.02257838,
        0.00335056,  0.00926289,  0.01060137,  0.00974614,  0.01660726,
        0.02419245,  0.01740896,  0.01219448, -0.02290199,  0.00146229,
        0.00698436,  0.01477018,  0.0078217 ,  0.01471581, -0.00550632,
        0.00461397, -0.00832996, -0.00469813,  0.01710154,  0.00727784,
        0.00888948,  0.00404979,  0.00151434,  0.00185668, -0.00637955,
        0.01601609, -0.00169102,  0.01152741, -0.00347789,  0.01187853,
        0.01338895,  0.00716166, -0.00413764, -0.0146584 ,  0.02226383,
       -0.00075295,  0.00193779,  0.01508522,  0.00872806,  0.01130567,
       -0.00775731,  0.009067  , -0.01349615,  0.01864646,  0.01165702,
        0.008611  , -0.00011907,  0.02201412,  0.01824271, -0.00

In [30]:
# lea과 shakira를 내적하는 코드
np.dot(lea_vector, shakira_vector)

0.43122327

- 1이 나와야 하는데 한참 낮은 수치인 0.43이 나왔다. factors를 늘리거나 iterations를 늘려야할 것 같다.
- 1이 나왔더라도 학습 데이터에 대해서 fitting은 되었지만 아직 보지 못한 데이터에 대한, 예를 들어 제가 queen을 선호할지에 대한 예측을 잘할지 못할지는 검증하지 않았기 때문에 아직 검증되지 않았다고 본다.

In [31]:
# 내 queen 에 대한 선호도 예측
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(lea_vector, queen_vector)

0.30367228

- 모델 기준은 만드는 사람이 정하기 나름이다. 따라서 객관적인 지표로 맏늘어서 기준을 정할 수도 있고 도메인 경험을 통해 정할 수도 있다.
- 따라서 객관적인 평가가 어려운 분야이다.

### 3. 비슷한 아티스트 찾기 및 유저에게 추천하기
#### 3-1. 비슷한 아티스트 찾기

In [32]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 0.1248163),
 (277, 0.122941405),
 (28, 0.12286004),
 (5, 0.121924564),
 (490, 0.12113562),
 (217, 0.12061732),
 (473, 0.12047776),
 (418, 0.120339535),
 (247, 0.11994894),
 (694, 0.11930737),
 (910, 0.117723696),
 (782, 0.11765072),
 (1018, 0.117458984),
 (55, 0.11709791),
 (268, 0.116436236)]

In [33]:
# (아티스트의 id, 유사도) Tuple 로 반환하고 있다. 아티스트의 id를 다시 아티스트의 이름으로 매핑
# artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'the killers',
 'red hot chili peppers',
 'oasis',
 'radiohead',
 'placebo',
 'u2',
 'the beatles',
 'foo fighters',
 'nirvana',
 'the white stripes',
 'the smashing pumpkins',
 'arctic monkeys',
 'pink floyd']

In [34]:
# 함수
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


In [35]:
get_similar_artist('2pac')

['2pac',
 'notorious b.i.g.',
 'the game',
 'snoop dogg',
 'nas',
 'dr. dre',
 'jay-z',
 '50 cent',
 'common',
 'ice cube']

#### 마니아
마니아들은 특정 장르의 아티스트들에게로 선호도가 집중되고, 다른 장르의 아티스트들과는 선호도가 낮게 나타난다. 이런 마니아들의 존재로 인해 같은 장르의 아티스트들의 벡터들도 더 가까워져서 getsimilarartist시 장르별 특성이 두드러지게 된다.

In [36]:
get_similar_artist('shakira')

['shakira',
 'christina aguilera',
 'kelly clarkson',
 'the pussycat dolls',
 'jennifer lopez',
 'gwen stefani',
 'black eyed peas',
 'p!nk',
 'natasha bedingfield',
 'no doubt']

#### 3-2. 유저에게 아티스트 추천하기
- AlternatingLeastSquares 클래스에 구현되어 있는 recommend 메서드 이용
- filter_already_liked_items 는 유저가 이미 평가한 아이템은 제외하는 Argument

In [37]:
user = user_to_idx['lea']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(382, 0.42494878),
 (627, 0.38864058),
 (550, 0.37277767),
 (409, 0.37020373),
 (355, 0.3695892),
 (350, 0.35846096),
 (24, 0.35655275),
 (724, 0.35359657),
 (28, 0.34690863),
 (901, 0.3444952),
 (944, 0.3406306),
 (5, 0.3401777),
 (391, 0.33883077),
 (2249, 0.33871236),
 (277, 0.33629),
 (503, 0.33546132),
 (354, 0.3332331),
 (369, 0.33118415),
 (159, 0.33082846),
 (1800, 0.32786685)]

In [38]:
[idx_to_artist[i[0]] for i in artist_recommended]

['mika',
 'maroon 5',
 'britney spears',
 'amy winehouse',
 'madonna',
 'rihanna',
 'jack johnson',
 'lily allen',
 'the killers',
 'pink',
 'avril lavigne',
 'red hot chili peppers',
 'christina aguilera',
 'katy perry',
 'muse',
 'green day',
 'nelly furtado',
 'justin timberlake',
 'dido',
 'lady gaga']

**AlternatingLeastSquares 클래스에 구현된 explain 메소드를 사용하면 내가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다.**

In [42]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

**method는 추천한 콘텐츠의 점수에 기여한 다른 콘텐츠와 기여도(합이 콘텐츠의 점수가 됩니다.)를 반환**

In [43]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('black eyed peas', 0.152513238748656),
 ('shakira', 0.09697231820652272),
 ('jason mraz', 0.05841548293484431),
 ('coldplay', 0.050315065180557575),
 ('maroon5', -1.5657560505056244e-05)]

**filter_already_liked_items=True로 했는데 앞에 maroon 5는 제가 이미 평가한 아이템이 나오고 있습니다.이것은 implicit 버전 0.4.2에서 생긴 버그**

https://danthetech.netlify.app/DataScience/evaluation-metrics-for-recommendation-system